# Sun Abundances

This code is scaffolded to help you calculate the stellar abundances for the Sun. This same code can be modified to help you calculate the abundances for the stars that are part of your project.

In [ ]:
#We will need several notebooks
import matplotlib.pyplot as plt #used to plot the spectra
import numpy as np #very useful for manipulating arrays
from astropy.io import fits #used to import fits files
from scipy.special import wofz #needed specifically for the voigt function
from scipy.optimize import curve_fit #a pretty good fitting function within the scipy module
import pymoog #this is used to calculate the stellar abundances
import pandas as pd #this is used to import tabulated data

In [ ]:
#Our ultimate goal is to measure the abundances of a star. We are going to start with something like the Sun.
#Our first step is to measure the EW of the spectral lines.

#define the functions that we might need

#You should define functions for anything you might use repeditavly.

#Voigt function
def voigt(x, x0, A, sigma, gamma):
    """
    Computes the Voigt line profile.

    Parameters
    ----------
    x : array_like
        The independent variable.
    sigma : float
        The standard deviation of the Gaussian component.
    gamma : float
        The HWHM of the Lorentzian component.

    Returns
    -------
    array_like
        The Voigt profile.
    """
    z = (((x-x0) + 1j * gamma) / (sigma * np.sqrt(2)))
    return 1 - A*np.real(wofz(z)) / (sigma * np.sqrt(2 * np.pi))

#Gaussian Function
def gaussian (x, x0, A, sigma):
    return 1 - A*np.exp(-(x-x0)**2/(2*sigma**2))

#Lorentzian function
def lorentzian (x, x0, A, gamma):
    return 1 - A*gamma**2/(np.pi*gamma**2 + ((x-x0)**2))

#gaussian area
def gaussian_area(A,sigma):
    return A*np.sqrt(np.pi*2)*sigma

In [ ]:
#Our first step will be to open the FITS file. You were provided a spectrum of the Sun. The file is titled 884752_melchiors_spectrum.fits
# The data is from Melchiors which is a library of science ready high-resolution spectra. https://www.royer.se/melchiors.html
#import the spectrum that you need
hdul = fits.open('884752_melchiors_spectrum.fits')
#this will print the information that is displayed in the FITS file.
hdul.info()

print(repr(hdul[0].header))
print(repr(hdul[1].header))

In [ ]:
spectrum_list = hdul[1].data #import all of the data
#for some reason they put the data in the format of a numpy array that contians lists. Which is very difficult to plot
# there is almost certainly a way to plot this better, but this is what I did.
#create a numpy array that is three by the length of the spectrum
norm_flux = np.zeros((3,len(spectrum_list)))

#do a for loop over the length of the spectrum that puts the wavelength, normalized spectrum, and uncertainty into
# a useful numpy array
for i in range(0,len(spectrum_list)):
    norm_flux[0,i] = spectrum_list[i][1]
    norm_flux[1,i] = spectrum_list[i][2]
    norm_flux[2,i] = spectrum_list[i][3]

In [ ]:
#This cell is the workhorse. You will plot a portion of the spectrum centered at a line you are interested in.
#   Then you will be able to fit a gaussian to the line and measure the area of the line.

line = 6726.666 #in angstroms
width = 10 #width of area you want to fit. Not in angstroms but in the spectral resolution (think pixels)

a = np.argmin(np.abs(norm_flux[0,:]-line)) #finding where the line is located in index space

# prompt: fit gaussian to data. We are going to use curve fit. We provide it with:
# -the function, we defined above (gaussian)
# - the wavelength
# - the flux,
# - the uncertainty in the flux (sigma)
# - an estimate of the initail conditions (p0)
# - and the bounds
#The output is the best fit values (popt) and the covariance matrix (pcov)
popt, pcov = curve_fit(gaussian, norm_flux[0,a-width:a+width], 
                       norm_flux[1,a-width:a+width], sigma=norm_flux[2,a-width:a+width], 
                       p0=[line, 0.4, 2], bounds=(0,[np.inf,1.0,np.inf]))

#taking the diagonal of the function and the square provides you with a
#   1-sigma estimate of the uncertainties for each fit parameter
perr = np.sqrt(np.diag(pcov)) 

print('Gaussian estimate of the line: ', popt)
print('and associated uncertainty:', perr)

#based on the best fit parameters, calculate what the fit line profile would be
output = gaussian (norm_flux[0,a-width:a+width], popt[0], popt[1], popt[2])

#Finally calculate the area of the gaussian using the best fit parameters
# we should probably propogate uncertainty here but we will save that for another day
print('Gaussian EW (ang): ',gaussian_area(popt[1],popt[2]))

plt.plot(norm_flux[0], norm_flux[1], '-', color='k') #the origional spectrum
plt.plot(norm_flux[0,a-width:a+width], output, '--', color='r') #the result of the gaussian fit
plt.plot((line, line), (0.1,0.15),'-',color='r',lw=2) #draw a read line where H-alpha is located
plt.xlabel('Wavelength (Ang)')
plt.ylabel('Normalized Flux')
plt.title('Fe line ' + str(line) + ' Ang')
plt.xlim(line-2,line+2)
#plt.ylim(0.3,1.05)
#plt.savefig('Fe_line_' + str(line) + '_Ang.png', dpi=300) #saving the figure with 300 dots per inch (resolution)
plt.show()
plt.close()

# Repeat the above cell 
until you have all of the equivalent Widths measured. Once you do, enter the eqvialent widths into the line_list_sun.txt file. Note that the EW need to be in units of milli-angstroms (mA). I encourage you to store these measurements in a spreadhseet or somewhere safe! This is your data. I have provided the first few set of lines. Please measure the EW to verify your results.

In [ ]:
#Load the line_list_sun.txt file that contains all of the equivalent widths you have entered.
line_list = pd.read_csv('line_list_example.txt', sep="\s+")
#line_list = pd.read_csv('line_list_sun.txt', sep="\s+")
print(line_list)

In [ ]:
#This runs pymoog! You will enter the line_list that you have provided 
#pymoog abfind
#                      Temperature [K], log(g) [cgs], [M/H], vmicro=microscopic velocity [0-2], linelist 
a = pymoog.abfind.abfind(5770, 4.4, 0, vmicro=1.5, line_list=line_list)
a.prepare_file()
a.run_moog()
a.read_output()

In [ ]:
#Print the results.
#The results are organized by element (atomic number).
a.abfind_res

In [ ]:
#plot the results
#you can change which element you are plotting by changing the atomic weight (element) below
element = 26

plt.plot(a.abfind_res[element]['EP'], a.abfind_res[element]['abund'],'o')
plt.plot((np.min(a.abfind_res[element]['EP']),np.max(a.abfind_res[element]['EP'])), 
          (np.average(a.abfind_res[element]['abund']),np.average(a.abfind_res[element]['abund'])), '--',color='r')
plt.xlabel('Excitation Potential')
plt.ylabel('Abundance')
plt.show()
plt.close()

plt.plot(a.abfind_res[element]['logRWin'], a.abfind_res[element]['abund'],'o')
plt.plot((np.min(a.abfind_res[element]['logRWin']),np.max(a.abfind_res[element]['logRWin'])), 
          (np.average(a.abfind_res[element]['abund']),np.average(a.abfind_res[element]['abund'])), '--',color='r')
plt.xlabel('Reduced Equivalent Width')
plt.ylabel('Abundance')
plt.show()
plt.close()

print('Average Abundance of ', element, ' = ', np.average(a.abfind_res[element]['abund']), 
      '+/-', np.std(a.abfind_res[element]['abund']))

In [ ]:
# calculate the average abundances for all of the elemental lines observed.
for element in a.abfind_res.keys():
    print('Average Abundance of ', element, ' = ', np.average(a.abfind_res[element]['abund']), 
          '+/-', np.std(a.abfind_res[element]['abund']))

# Change the fundamental parameters
Go back to the cell where you ran moog. Change the fundamental parameters 

# Assess the lines, are all of these lines good lines?
Remember this is real data! Individual lines can be affected by blending with other lines, having noise or artifiacts from the Earths atmosphere. That is why we use multiple lines. We may have to throw out a line if we believe it is not behaving like it should! 

# Compare your results to the accepted abundances of the Sun